In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


/home/agomotto3000/anaconda3/envs/mase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


In [2]:
batch_size = 512
model_name = "toy_convnet"
dataset_name = "Cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [4]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    #if get_mase_op(node) == 'conv2d':
    if node.op == "call_module":
        #print(node.name)
        #print(50*'-')
        #print(node.target)
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #pprint(mg.modules[node.target].weight)
        #pprint(mg.modules[node.target].state_dict())
        #pprint(mg.mo)
        print(50*'-')
print(mg.modules)
#print(type(mg.model))

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
{'': GraphModule(
  (block_1): Module(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block_2): Module(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (block_3): Module(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU(

In [5]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
    activation_pruning_pass,
)
pass_args = {
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)



KeyError: 'weight'

In [ ]:
print(mg.model.state_dict())

OrderedDict([('block_1.0.bias', tensor([-0.0150,  0.1194, -0.0869,  0.1154, -0.1577, -0.0850,  0.0252,  0.1689])), ('block_1.0.parametrizations.weight.original', tensor([[[[ 0.0482, -0.1878,  0.1399],
          [-0.0447, -0.1207,  0.0728],
          [ 0.1810,  0.1505, -0.0590]],

         [[ 0.0790,  0.0771,  0.1427],
          [-0.0824,  0.0353,  0.0553],
          [ 0.1773, -0.0354, -0.0334]],

         [[ 0.0501,  0.1248,  0.1277],
          [ 0.0594,  0.0126, -0.0937],
          [ 0.1721,  0.0603,  0.1783]]],


        [[[-0.1910,  0.0375, -0.1549],
          [-0.1783,  0.1765, -0.1179],
          [-0.1074,  0.0089, -0.0264]],

         [[ 0.0115, -0.1628,  0.0479],
          [ 0.1017,  0.1554,  0.1056],
          [ 0.0324, -0.0862, -0.0767]],

         [[ 0.0320, -0.1785,  0.0332],
          [-0.1306,  0.0354,  0.1682],
          [-0.0375,  0.0699,  0.1177]]],


        [[[ 0.0533,  0.0707, -0.1811],
          [ 0.0834,  0.0610, -0.0461],
          [ 0.0318, -0.0468, -0.0558]],

 

In [ ]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    #if get_mase_op(node) == 'linear':
    if node.op == "call_module":
        #print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        #pprint(node.meta['mase'].parameters['common']['args'])
        pprint(mg.modules[node.target]._forward_pre_hooks.items())
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
       # print(mg.model.state_dict())
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #print(mg.modules[node.target].parametrizations['weight'][0].mask == mg.modules[node.target].parametrizations['weight'][1].mask)
        # total_w = 0
        # pruned_w = 0
        # w = mg.modules[node.target].weight
        # for s in w:
        #     total_w += s.numel()
        #     pruned_w += s.numel() - s.nonzero().numel()

        # pruned_percent = pruned_w / total_w
        # print(f"Pruned percent: {pruned_percent}")

        # print(50*'-')


odict_items([(10, <function get_activation_hook.<locals>.sparsify_input at 0x7f651a1e97e0>)])
odict_items([])
odict_items([])
odict_items([(11, <function get_activation_hook.<locals>.sparsify_input at 0x7f651544e8c0>)])
odict_items([])
odict_items([(12, <function get_activation_hook.<locals>.sparsify_input at 0x7f651544f2e0>)])
odict_items([])
odict_items([(13, <function get_activation_hook.<locals>.sparsify_input at 0x7f651544f910>)])
odict_items([])
odict_items([])
odict_items([(14, <function get_activation_hook.<locals>.sparsify_input at 0x7f651544f880>)])


In [ ]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        pprint(node.meta['mase'].parameters['common']['args'])
        #pprint(mg.modules[node.target].weight)
        # pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

        # total_w = 0
        # pruned_w = 0
        # mask_2= mg.modules[node.target].parametrizations['weight'][0].mask
        # for s in mask_2:
        #     total_w += s.numel()
        #     pruned_w += s.numel() - s.nonzero().numel()

        # pruned_percent = pruned_w / total_w
        # print(f"Pruned percent: {pruned_percent}")

        print(50*'-')
#print(mg.model.state_dict())

Layer: seq_blocks_2
{'bias': {'from': None,
          'precision': [32],
          'shape': [5],
          'type': 'float',
          'value': Parameter containing:
tensor([-0.2035,  0.1072,  0.2207,  0.2167,  0.0330], requires_grad=True)},
 'data_in_0': {'precision': [32],
               'shape': [512, 16],
               'torch_dtype': torch.float32,
               'type': 'float',
               'value': tensor([[0.0000, 0.6918, 0.1922,  ..., 0.0000, 0.2194, 0.4817],
        [0.0000, 0.3129, 0.0933,  ..., 0.0000, 0.0000, 0.0000],
        [0.5225, 0.2079, 0.1787,  ..., 0.0000, 0.1940, 0.0000],
        ...,
        [0.4002, 0.3301, 1.0308,  ..., 0.0000, 1.3613, 0.0000],
        [0.0000, 0.4540, 0.2304,  ..., 0.0000, 0.1260, 0.0000],
        [0.0000, 0.6719, 1.6635,  ..., 0.5214, 1.6486, 0.0000]],
       grad_fn=<ReluBackward0>)},
 'weight': {'from': None,
            'precision': [32],
            'shape': [5, 16],
            'type': 'float',
            'value': Parameter containing

In [ ]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
)
pass_args = {
    "weight":{
        "scope" : "global",
        "granularity" : "elementwise",
        "method" :  "l1-norm",
        "sparsity" : 0.6,
    },
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.6,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)


OrderedDict([('seq_blocks.0.weight', tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])), ('seq_blocks.0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])), ('seq_blocks.0.running_mean', tensor([-0.0050, -0.0008,  0.0117,  0.0123,  0.0122,  0.0126,  0.0009,  0.0042,
         0.0009,  0.0062,  0.0059,  0.0099,  0.0060,  0.0104,  0.0113,  0.0112])), ('seq_blocks.0.running_var', tensor([1.0098, 1.0104, 1.0108, 1.0118, 1.0102, 1.0118, 0.9849, 0.9822, 0.9849,
        0.9984, 1.0017, 1.0007, 0.9980, 0.9983, 1.0123, 1.0123])), ('seq_blocks.0.num_batches_tracked', tensor(2)), ('seq_blocks.2.bias', tensor([-0.2035,  0.1072,  0.2207,  0.2167,  0.0330])), ('seq_blocks.2.parametrizations.weight.original', tensor([[ 0.0280,  0.1994,  0.1990, -0.0459,  0.1669,  0.2383,  0.2267, -0.1722,
          0.2401,  0.0899,  0.2032, -0.2431, -0.0539,  0.0350, -0.0878,  0.0864],
        [-0.2471,  0.1335, -0.0760,  0.0635,  0.2423,  0.0355,  0.1517,  0.0415,
 

In [ ]:
print(mg.model.state_dict())
#print(node.meta['mase'].parameters['common']['args']['weight']['value'])

OrderedDict([('block_1.0.bias', tensor([-0.0150,  0.1194, -0.0869,  0.1154, -0.1577, -0.0850,  0.0252,  0.1689])), ('block_1.0.parametrizations.weight.original', tensor([[[[ 0.0482, -0.1878,  0.1399],
          [-0.0447, -0.1207,  0.0728],
          [ 0.1810,  0.1505, -0.0590]],

         [[ 0.0790,  0.0771,  0.1427],
          [-0.0824,  0.0353,  0.0553],
          [ 0.1773, -0.0354, -0.0334]],

         [[ 0.0501,  0.1248,  0.1277],
          [ 0.0594,  0.0126, -0.0937],
          [ 0.1721,  0.0603,  0.1783]]],


        [[[-0.1910,  0.0375, -0.1549],
          [-0.1783,  0.1765, -0.1179],
          [-0.1074,  0.0089, -0.0264]],

         [[ 0.0115, -0.1628,  0.0479],
          [ 0.1017,  0.1554,  0.1056],
          [ 0.0324, -0.0862, -0.0767]],

         [[ 0.0320, -0.1785,  0.0332],
          [-0.1306,  0.0354,  0.1682],
          [-0.0375,  0.0699,  0.1177]]],


        [[[ 0.0533,  0.0707, -0.1811],
          [ 0.0834,  0.0610, -0.0461],
          [ 0.0318, -0.0468, -0.0558]],

 

In [ ]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [ ]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op


# # pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(node.name)
        print(50*'-')
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        #pprint(mg.modules[node.target].weight)
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

        print(50*'-')

seq_blocks_2
--------------------------------------------------
Parameter containing:
tensor([[ 0.0280,  0.1994,  0.1990, -0.0459,  0.1669,  0.2383,  0.2267, -0.1722,
          0.2401,  0.0899,  0.2032, -0.2431, -0.0539,  0.0350, -0.0878,  0.0864],
        [-0.2471,  0.1335, -0.0760,  0.0635,  0.2423,  0.0355,  0.1517,  0.0415,
         -0.1401,  0.0164, -0.2077,  0.0701,  0.0952, -0.0982,  0.2382,  0.1787],
        [-0.0695, -0.1765,  0.2361, -0.0543,  0.0771,  0.2328,  0.1437,  0.0289,
          0.0836,  0.1229,  0.2050, -0.1231, -0.1486,  0.2418, -0.1702,  0.0644],
        [-0.1972, -0.2393, -0.1072, -0.0610, -0.0949, -0.2070,  0.1950, -0.2175,
         -0.2453, -0.0608,  0.1447, -0.1106,  0.0976,  0.0070,  0.1467,  0.2468],
        [-0.2423, -0.1813, -0.1228, -0.0613, -0.0382, -0.1995,  0.1495, -0.1170,
         -0.1918, -0.1813,  0.1138, -0.1162, -0.1512, -0.0801,  0.2345, -0.1161]],
       requires_grad=True)
--------------------------------------------------
